### ¿Qué hace este script?

Calcula para cada dispositivo de red(AP) y rango de fecha, el tiempo promedio de sesión (usuarios.tiempoPromedioSesionSitio)

In [374]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import random
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [375]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
)

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [376]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertará e indice para control de ejecución

In [377]:
indice = parametros.usuarios_tablero07_index
indice_control = parametros.tableros_mintic_control

### Funcion para JSON compatible con ElasticSearch

In [378]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [379]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["usuarios.Tablero07.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_usuarios_sesiones_sitio"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.Tablero07.fechaControl']
except:
    pass
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-07-01 00:50:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:
* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, id_Beneficiario).

In [380]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [381]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [382]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacía o no valida

In [383]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.location"]=='')].index)

In [384]:
datos_semilla

,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,site_id,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,usuarios.idBeneficiario,usuarios.location
0,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469"
1,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667"
2,C. E. R. MERCEDES YEPES,CO-ANT,VDA. LA MESETA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18787,"6.93848274,-75.151177190"
3,C. E. R. MEDIAS FALDAS,CO-ANT,VDA EL LIMÓN,,ANTIOQUIA,TIENE ENERGÍA,ANORÍ,18802,"7.04797951,-75.19442611"
4,C. E. R. ROSAURA ORTEGA,CO-ANT,BELLAVISTA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18804,"7.06272584,-75.22730972"
...,...,...,...,...,...,...,...,...,...
6941,ESCUELA RURAL SAN MIGUEL,CO-VAU,COMUNIDAD SAN MIGUEL,,VAUPÉS,SOLUCIÓN SOLAR,PACOA,67732,"0.11678,-70.072640000"
6942,ESCUELA SANTA ISABEL,CO-VAU,COMUNIDAD SANTA ISABEL,,VAUPÉS,SOLUCIÓN SOLAR,PACOA,67733,"6.39026335,-73.138349330"
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,CO-VAU,COMUNIDAD DE SAN LUIS,,VAUPÉS,SIN INFORMACIÓN,PACOA,70622,"0.08164385,-70.07416384"
6945,ESCUELA RURAL DE TOAKA,CO-VAU,COMUNIDAD DE TOAKA,,VAUPÉS,SIN INFORMACIÓN,PACOA,70624,"1.007644476,-71.25103723"


### leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR
* site_id para cruzar con las misma llave de semilla.
* datos del dispositivo: mac, status, ip.
* ap_group para identificar si la conexión es indoor/outdoor

In [385]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe.

In [386]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

Se corrigen datos de ap group con formato no valido

In [387]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos con site_id y mac unicos.

In [388]:
#datos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)
datos_dev = datos_dev.drop_duplicates('mac')

In [389]:
datos_dev

,ap_group,ip,site_id,mac,status
0,INDOOR,172.28.118.179,44745-ZGYO227,BC:E6:7C:5E:66:88,online
1,OUTDOOR,172.28.118.180,44745-ZGYO227,58:C1:7A:E9:C9:6F,online
2,OUTDOOR,172.28.118.181,44745-ZGYO227,58:C1:7A:E9:C8:5C,online
3,INDOOR,172.28.118.171,44747-ZGYO413,BC:E6:7C:ED:F2:4F,online
4,OUTDOOR,172.28.118.172,44747-ZGYO413,BC:E6:7C:4F:07:88,online
...,...,...,...,...,...
4498,INDOOR,172.25.30.163,49318-ZZZY605,BC:E6:7C:EC:AA:C7,offline
4499,OUTDOOR,172.25.84.76,37717-ZZZY751,BC:A9:93:0C:7B:9D,offline
4500,OUTDOOR,172.25.84.77,37717-ZZZY751,BC:A9:93:0C:E3:1D,offline
4501,INDOOR,172.25.38.243,47123-ZGYO420,BC:E6:7C:ED:8A:EA,online


Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario 

In [390]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

### Se lee información de Ohmyfi consumos

Apartir de esta lectura se toma el valor del tiempo promedio de sesión en minutos. Se toma de referencia los campos:
* fecha_inicio a partir de la cual se calcula fecha control
* tiempo_sesion_minutos
* mac_usuario 
* lugar_cod

In [391]:
def traeSesiones(fecha_max,fecha_tope):
    total_docs = 500000
    response = es.search(
        index= parametros.ohmyfi_consumos_index,
        body={
                  "_source": ["lugar_cod", "tiempo_sesion_minutos","mac_ap","fecha_inicio"]
                , "query": {
                      "range": {
                            "fecha_inicio": {
                            "gte": fecha_max,
                            "lt": fecha_tope
                            }
                        }
                  }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 


In [392]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Se ejecuta consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [393]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

if datos_consumos is None or datos_consumos.empty:
    while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [394]:
type(fecha_max_mintic)

str

In [395]:
type(fecha_ejecucion)

str

In [396]:
datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

In [397]:
datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In [398]:
datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                             ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                             ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                             , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

In [399]:
fecha_max_mintic

'2021-07-01 00:50:00'

In [400]:
datos_det_conex

,usuarios.fechaConexionUsuarios,mac_usuario,usuarios.tipoDispositivoUsuarios,lugar,documento,tipodoc,site_id,fecha_control,usuarios.sistemaOperativoUsuarios,fecha
0,2021-07-01 01:00:14,88:10:8F:FB:93:81,Smartphone,CGTO DE VALENCIA DE JESUS_VALLEDUPAR,10674528,TI,67697-ZGYO849,2021-07-01 01:00:00,Android,2021-07-01
1,2021-07-01 01:00:35,68:A0:3E:98:AA:CF,Smartphone,GUAIMARITO_HATONUEVO,1006734120,CC,37617-ZZZY740,2021-07-01 01:00:00,Android,2021-07-01
3,2021-07-01 01:00:41,88:AD:D2:5E:95:18,Smartphone,TOLIMA_TUCHIN,1193557935,CC,32313-ZGYO438,2021-07-01 01:00:00,Android,2021-07-01
4,2021-07-01 01:00:44,EC:AA:25:29:C4:89,Smartphone,"SACANA ABAJO, MOMIL",1003213476,CC,31730-ZGYO298,2021-07-01 01:00:00,Android,2021-07-01
9,2021-07-01 01:01:55,B8:94:36:8C:37:98,Smartphone,"DE BREMEN, MORROA",64740741,CC,47362-ZZZY006,2021-07-01 01:00:00,Android,2021-07-01
10,2021-07-01 01:30:06,D0:FC:CC:17:05:9A,Smartphone,CALLE 115 # 06 - 60_TURBO,39310107,CC,10835-ZGYO372,2021-07-01 01:30:00,Android,2021-07-01
11,2021-07-01 01:30:34,7C:38:AD:69:6B:5B,Smartphone,VILLANUEVA_PUERTO LIBERTADOR,1145678904,CC,32066-ZZZY759,2021-07-01 01:30:00,Android,2021-07-01
13,2021-07-01 01:31:41,9C:28:F7:A0:B9:D0,Smartphone,SANTA ANA_ITUANGO,1037262939,TI,20270-ZGYO186,2021-07-01 01:30:00,Android,2021-07-01


In [401]:
datos_det_conex.columns

Index(['usuarios.fechaConexionUsuarios', 'mac_usuario',
       'usuarios.tipoDispositivoUsuarios', 'lugar', 'documento', 'tipodoc',
       'site_id', 'fecha_control', 'usuarios.sistemaOperativoUsuarios',
       'fecha'],
      dtype='object')

# Escribiendo en indice la información de tiempo promedio sesión en sitio

In [402]:
use_these_keys = ['usuarios.tiempoPromedioSesionSitio'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.localidad'
                  , 'usuarios.siteID'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.location'
                  , 'usuarios.macRed'
                  , 'usuarios.apGroup'
                  , 'usuarios.usuariosConectados'
                  , 'usuarios.sesiones_Usuarios'
                  , 'usuarios.fechaControl'
                  , 'usuarios.fecha'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.hora'
                  , 'usuarios.minuto'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']
def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            document = document.rename(index={'usuarios.tiempoPromedioSesionSitio_x': 'usuarios.tiempoPromedioSesionSitio','usuarios.macRed_x':'usuarios.macRed','usuarios.apGroup_x':'usuarios.apGroup','usuarios.fechaControl_x':'usuarios.fechaControl'})
            yield {
                    "_index": indice, 
                    "_id": f"{str(document['usuarios.siteID']) + '-' + str(document['usuarios.fechaControl']) + '-' + str(document['usuarios.macRed'])+'-'+str(random.randrange(10000000))}",
                    "_source": filterKeys(document),
                }
            

In [403]:
use_these_keys

['usuarios.tiempoPromedioSesionSitio',
 'usuarios.nomCentroDigital',
 'usuarios.codISO',
 'usuarios.idBeneficiario',
 'usuarios.localidad',
 'usuarios.siteID',
 'usuarios.nombreDepartamento',
 'usuarios.sistemaEnergia',
 'usuarios.nombreMunicipio',
 'usuarios.location',
 'usuarios.macRed',
 'usuarios.apGroup',
 'usuarios.usuariosConectados',
 'usuarios.sesiones_Usuarios',
 'usuarios.fechaControl',
 'usuarios.fecha',
 'usuarios.anyo',
 'usuarios.mes',
 'usuarios.dia',
 'usuarios.hora',
 'usuarios.minuto',
 'nombreDepartamento',
 'nombreMunicipio',
 'idBeneficiario',
 'fecha',
 'anyo',
 'mes',
 'dia',
 '@timestamp']

Se agrupa por lugar_cod, fecha_control, mac_usuario y se promedia el tiempo_sesion_minutos. Este genera el campo del indice final:
* usuarios.tiempoPromedioSesionSitio

In [404]:
datos_consumos.columns

Index(['fecha_inicio', 'tiempo_sesion_minutos', 'mac_ap', 'lugar_cod'], dtype='object')

In [405]:
datos_consumos

,fecha_inicio,tiempo_sesion_minutos,mac_ap,lugar_cod
0,2021-07-01 00:52:47,5.00,58:C1:7A:E8:76:8B,22483-ZGYO057
1,2021-07-01 00:57:48,0.78,58:C1:7A:E8:77:EA,22483-ZGYO057
2,2021-07-01 00:58:34,9.97,58:C1:7A:E8:76:8B,22483-ZGYO057
3,2021-07-01 00:57:24,15.63,BC:E6:7C:EF:FC:D1,30731-ZGYO144
4,2021-07-01 00:52:16,0.35,BC:E6:7C:E7:FD:51,70813-ZZZY057
...,...,...,...,...
492,2021-07-01 01:20:13,102.22,BC:E6:7C:EF:FE:4F,30435-ZGYO116
493,2021-07-01 01:20:29,21.28,BC:E6:7C:EF:FE:4F,30435-ZGYO116
494,2021-07-01 01:01:29,4.40,BC:E6:7C:E7:FC:FB,47054-VZVF187
495,2021-07-01 01:06:06,3.55,BC:E6:7C:E7:FC:FB,47054-VZVF187


 Se corrigen valores errados de site_id en detalle conexiones

In [406]:
try:
    datos_consumos = datos_consumos.rename(columns={'lugar_cod' :'site_id'})
    datos_consumos['mac_ap'] = datos_consumos['mac_ap'].str.replace('-',':')
    datos_consumos['fecha_control'] = datos_consumos["fecha_inicio"].str[0:-4] + '0:00'
    tiempoPromedioSesionSitio=datos_consumos[['site_id','mac_ap','fecha_control','tiempo_sesion_minutos']].groupby(['site_id','mac_ap','fecha_control']).agg(['mean']).reset_index()
    tiempoPromedioSesionSitio.columns = tiempoPromedioSesionSitio.columns.droplevel(1)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'tiempo_sesion_minutos' : 'usuarios.tiempoPromedioSesionSitio'})
    tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'] = round(tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'],6)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'mac_ap' : 'usuarios.macRed'
                                                                          ,'fecha_control' : 'usuarios.fechaControl'})
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio,datos_semilla, on ='site_id', how='inner')
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio, datos_dev, on=['site_id','usuarios.macRed'], how='left')
    tiempoPromedioSesionSitio.fillna({'usuarios.apGroup':'No identificado'},inplace=True)

    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento']].groupby(["fecha_control","site_id"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})

    usuariosConectados = datos_det_conex[["fecha_control","site_id","mac_usuario"]].groupby(["fecha_control","site_id"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.usuariosConectados' })
    usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id"], how='inner')
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    usuariosConectados = pd.merge(usuariosConectados, tiempoPromedioSesionSitio, on=['site_id'], how='inner')
    usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                              ,'usuarios.consumoUsuariosDescarga':0
                              ,'usuarios.consumoUsuariosCarga':0},inplace=True)

    usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    usuariosConectados['usuarios.usuariosConectados'] = usuariosConectados['usuarios.usuariosConectados'].astype(int)
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})

    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'site_id' : 'usuarios.siteID'})
    tiempoPromedioSesionSitio["usuarios.fecha"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str[0:10]
    tiempoPromedioSesionSitio["usuarios.anyo"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[0:4]
    tiempoPromedioSesionSitio["usuarios.mes"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[5:7]
    tiempoPromedioSesionSitio["usuarios.dia"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[8:10]
    tiempoPromedioSesionSitio["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    tiempoPromedioSesionSitio["usuarios.minuto"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]

    tiempoPromedioSesionSitio['nombreDepartamento'] = tiempoPromedioSesionSitio['usuarios.nombreDepartamento']
    tiempoPromedioSesionSitio['nombreMunicipio'] = tiempoPromedioSesionSitio['usuarios.nombreMunicipio']
    tiempoPromedioSesionSitio['idBeneficiario'] = tiempoPromedioSesionSitio['usuarios.idBeneficiario']
    tiempoPromedioSesionSitio['fecha'] = tiempoPromedioSesionSitio['usuarios.fecha']
    tiempoPromedioSesionSitio['anyo'] = tiempoPromedioSesionSitio['usuarios.anyo']
    tiempoPromedioSesionSitio['mes'] = tiempoPromedioSesionSitio['usuarios.mes']
    tiempoPromedioSesionSitio['dia'] = tiempoPromedioSesionSitio['usuarios.dia']
    tiempoPromedioSesionSitio['@timestamp'] = now.isoformat()

    usuariosConectados2 = pd.merge(tiempoPromedioSesionSitio, usuariosConectados, on=['usuarios.siteID'], how='inner')  
    usuariosConectados2

    salida = helpers.bulk(es, doc_generator(usuariosConectados2))
    
    print("Fecha: ", now,"- Tiempo promedio sesion en sitio insertado en indice principal:",salida[0])
    
except:
    print("Fecha: ", now,"- Nada para insertar en indice principal")

Fecha:  2021-08-02 22:05:02.309076 - Tiempo promedio sesion en sitio insertado en indice principal: 33


In [407]:
usuariosConectados2

,usuarios.siteID,usuarios.macRed_x,usuarios.fechaControl_x,usuarios.tiempoPromedioSesionSitio_x,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,...,usuarios.codISO_y,usuarios.localidad_y,usuarios.nombreDepartamento_y,usuarios.sistemaEnergia_y,usuarios.nombreMunicipio_y,usuarios.idBeneficiario_y,usuarios.location_y,usuarios.apGroup_y,ip_y,status_y
0,10835-ZGYO372,BC:E6:7C:4E:3E:8F,2021-07-01 01:30:00,7.850000,E.U. SIETE DE AGOSTO,CO-ANT,CALLE 115 # 06 - 60,ANTIOQUIA,RED INTERCONECTADA,TURBO,...,CO-ANT,CALLE 115 # 06 - 60,ANTIOQUIA,RED INTERCONECTADA,TURBO,10835,"10.88307108,-75.077732850",OUTDOOR,172.28.118.244,online
1,20270-ZGYO186,BC:E6:7C:4E:3C:DE,2021-07-01 01:30:00,120.000000,INSTITUCIÓN EDUCATIVA RURAL SANTA ANA,CO-ANT,SANTA ANA,ANTIOQUIA,RED INTERCONECTADA,ITUANGO,...,CO-ANT,SANTA ANA,ANTIOQUIA,RED INTERCONECTADA,ITUANGO,20270,"8.051666667,-75.669000000",OUTDOOR,172.28.107.197,online
2,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:00:00,0.966667,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
3,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:00:00,0.966667,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
4,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:00:00,0.966667,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
5,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:00:00,0.966667,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
6,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:10:00,0.675000,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
7,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:10:00,0.675000,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
8,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:10:00,0.675000,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online
9,31730-ZGYO298,BC:E6:7C:E7:FC:D8,2021-07-01 01:10:00,0.675000,ESC NUEVA SACANA ABAJO,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,...,CO-COR,SACANA ABAJO,CÓRDOBA,RED INTERCONECTADA,MOMIL,31730,"9.32313135,-75.622755310",OUTDOOR,172.25.39.220,online


In [408]:
usuariosConectados2.columns

Index(['usuarios.siteID', 'usuarios.macRed_x', 'usuarios.fechaControl_x',
       'usuarios.tiempoPromedioSesionSitio_x', 'usuarios.nomCentroDigital',
       'usuarios.codISO', 'usuarios.localidad', 'usuarios.nombreDepartamento',
       'usuarios.sistemaEnergia', 'usuarios.nombreMunicipio',
       'usuarios.idBeneficiario', 'usuarios.location', 'usuarios.apGroup_x',
       'ip_x', 'status_x', 'usuarios.fecha', 'usuarios.anyo', 'usuarios.mes',
       'usuarios.dia', 'usuarios.hora', 'usuarios.minuto',
       'nombreDepartamento', 'nombreMunicipio', 'idBeneficiario', 'fecha',
       'anyo', 'mes', 'dia', '@timestamp', 'usuarios.nomCentroDigital_x',
       'usuarios.codISO_x', 'usuarios.localidad_x',
       'usuarios.nombreDepartamento_x', 'usuarios.sistemaEnergia_x',
       'usuarios.nombreMunicipio_x', 'usuarios.idBeneficiario_x',
       'usuarios.location_x', 'usuarios.fechaControl_y',
       'usuarios.usuariosConectados', 'usuarios.sesiones_Usuarios',
       'usuarios.macRed_y', 'usuar

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [409]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_usuarios_sesiones_sitio',
        body = { 'jerarquia_usuarios_sesiones_sitio': 'usuarios_sesiones_sitio','usuarios.Tablero07.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-07-01 01:40:00
